In [50]:
!pip install ddf_utils
import pandas as pd
import xlrd 
import os
from ddf_utils.str import to_concept_id, format_float_digits
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from functools import partial

In [5]:
sheet_loader = partial(pd.read_excel, skiprows=2)

In [6]:
number_formatter = partial(format_float_digits, digits=8)

In [7]:
source_file = 'bp-stats-review-2020-all-data.xlsx'

In [8]:
sheets = xlrd.open_workbook(source_file, on_demand=True).sheet_names()

In [9]:
sheets

['Contents',
 'Primary Energy Consumption',
 'Primary Energy - Cons by fuel',
 'Primary Energy - Cons capita',
 'Carbon Dioxide Emissions',
 'Oil - Proved reserves',
 'Oil - Proved reserves history',
 'Oil Production - Barrels',
 'Oil Production - Tonnes',
 'Oil Production - Crude Conds',
 'Oil Production - NGLs',
 'Total Liquids - Consumption',
 'Oil Consumption - Barrels',
 'Oil Consumption - Tonnes',
 'Oil Consumption - EJ',
 'Oil - Regional Consumption ',
 'Oil - Spot crude prices',
 'Oil - Crude prices since 1861',
 'Oil - Refinery throughput',
 'Oil - Refining capacity',
 'Oil - Regional refining margins',
 'Oil - Trade movements',
 'Oil - Inter-area movements ',
 'Oil - Trade 2018 - 2019',
 'Gas - Proved reserves',
 'Gas - Proved reserves history ',
 'Gas Production - Bcm',
 'Gas Production - Bcf',
 'Gas Production - EJ',
 'Gas Consumption - Bcm',
 'Gas Consumption - Bcf',
 'Gas Consumption - EJ',
 'Gas - Prices ',
 'Gas - Inter-regional trade',
 'Gas - LNG imports',
 'Gas - LNG

In [10]:
contents = pd.read_excel(source_file, sheet_name='Contents', header=None)

In [11]:
contents

,0
0,NaN
1,bp Statistical Review of World Energy June 2020
2,NaN
3,This workbook contains information presented ...
4,"bp Statistical Review of World Energy, which ..."
...,...
99,Approximate conversion factors
100,NaN
101,Definitions
102,NaN


In [12]:
tabs_to_parse = contents.loc[6:82, 0]

In [13]:
tabs_to_parse

6                                                   NaN
7                   http://www.bp.com/statisticalreview
8                                                   NaN
9     Please use the contents or the tabs at the bot...
10                                                  NaN
                            ...                        
78    Renewables - Biofuels production - Petajoules ...
79    Renewables - Biofuels consumption - Kboe/d (fr...
80    Renewables - Biofuels consumption - Petajoules...
81                                                  NaN
82             Electricity generation - TWh (from 1985)
Name: 0, Length: 77, dtype: object

In [14]:
len(sheets[1:-2])

77

In [15]:
tabs = pd.DataFrame({'full_name': tabs_to_parse, 'tab_name': sheets[1:-2]})

In [16]:
tabs

,full_name,tab_name
6,NaN,Primary Energy Consumption
7,http://www.bp.com/statisticalreview,Primary Energy - Cons by fuel
8,NaN,Primary Energy - Cons capita
9,Please use the contents or the tabs at the bot...,Carbon Dioxide Emissions
10,NaN,Oil - Proved reserves
...,...,...
78,Renewables - Biofuels production - Petajoules ...,Cobalt and Lithium - Prices
79,Renewables - Biofuels consumption - Kboe/d (fr...,Geothermal Capacity
80,Renewables - Biofuels consumption - Petajoules...,Solar Capacity
81,NaN,Wind Capacity


In [17]:
tabs.to_csv('tabs.csv', index=False)

In [18]:
def preprocess(data):
    """preprocessing the data:
    1. rename the first column to geo_name
    2. rename the geo_name to alphanumeric
    3. drop all empty lines and lines after 'total world'

    Note: This function only applies to the tab with country as row index
    and year as column index.
    """
    data = data.rename(columns={data.columns[0]: 'geo_name'})
    data['geo'] = data['geo_name'].map(to_concept_id)
    data = data.set_index('geo')
    data = data.dropna(how='all')
    data = data.loc[:'total_world']
    data = data.reset_index()
    return data

In [19]:
def apply_map(x, m):
    """general function to replace value in a Series."""
    if x in m:
        return m[x]
    return x

In [20]:
def process_1(data, ddf_id):
    data = data.dropna(axis=1, how='all')
    data = data.drop('geo_name', axis=1)
    idx = list(data.columns).index(2019)  #TODO: change the year column
    data = data.iloc[:, :idx + 1]  # drop columns after latest year of each sheet.
    #data = data.drop(['2019.1', '2019.2', '2008-18'], axis=1)
    
    data = data.set_index('geo')

    d = data.T.unstack()
    d = d.dropna()
    d = d.reset_index()
    d.columns = ['geo', 'year', ddf_id]
    d[ddf_id] = d[ddf_id].map(number_formatter)

    return d.sort_values(by=['geo', 'year'])

In [21]:
tabs.iloc[0]

full_name                           NaN
tab_name     Primary Energy Consumption
Name: 6, dtype: object

In [22]:
data1 = sheet_loader(source_file, sheet_name='Primary Energy Consumption')

In [23]:
data1 = preprocess(data1)

In [24]:
data1.head()

,geo,geo_name,1965,1966,1967,1968,1969,1970,1971,1972,...,2015,2016,2017,2018,2019,2019.1,2008-18,2019.2,Unnamed: 59,Unnamed: 60
0,canada,Canada,4.924317,5.239095,5.480214,5.836056,6.185953,6.624581,6.831992,7.348642,...,13.993650,13.936678,14.109482,14.349534,14.214058,-0.009441,0.006404,0.024343,NaN,NaN
1,mexico,Mexico,1.050253,1.111572,1.132470,1.232997,1.353689,1.444834,1.519984,1.680466,...,7.688300,7.785419,7.895341,7.833322,7.720742,-0.014372,0.009057,0.013223,NaN,NaN
2,us,US,52.433795,55.383038,57.311776,60.799094,64.049612,66.215508,67.591944,71.064330,...,92.149004,92.017799,92.330525,95.602919,94.648804,-0.009980,0.001054,0.162098,NaN,NaN
3,total_north_america,Total North America,58.408365,61.733704,63.924460,67.868147,71.589254,74.284923,75.943920,80.093438,...,113.830955,113.739897,114.335348,117.785775,116.583604,-0.010206,0.002204,0.199664,NaN,NaN
4,argentina,Argentina,1.127632,1.165108,1.199725,1.241234,1.283912,1.212762,1.288378,1.311442,...,3.585803,3.576195,3.566920,3.540866,3.462769,-0.022056,0.012502,0.005930,NaN,NaN


In [26]:
process_1(data1, to_concept_id('Primary Energy: Consumption - Mtoe')).head(100)

,geo,year,primary_energy_consumption_mtoe
3520,algeria,1965,0.08913947
3521,algeria,1966,0.10716806
3522,algeria,1967,0.10142615
3523,algeria,1968,0.10887872
3524,algeria,1969,0.12050818
...,...,...,...
260,argentina,2005,2.802798
261,argentina,2006,2.94482844
262,argentina,2007,3.07479095
263,argentina,2008,3.12715637


In [27]:
df = process_1(data1, to_concept_id('Primary Energy: Consumption - Mtoe'))

In [28]:
data1['geo_name'].unique()

array(['Canada', 'Mexico', 'US', 'Total North America', 'Argentina',
       'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Peru',
       'Trinidad & Tobago', 'Venezuela', 'Central America',
       'Other Caribbean', 'Other South America',
       'Total S. & Cent. America', 'Austria', 'Belgium', 'Bulgaria',
       'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia',
       'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland',
       'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
       'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal',
       'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
       'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom',
       'Other Europe', 'Total Europe', 'Azerbaijan', 'Belarus',
       'Kazakhstan', 'Russian Federation', 'Turkmenistan', 'USSR',
       'Uzbekistan', 'Other CIS', 'Total CIS', 'Iran', 'Iraq', 'Israel',
       'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates',
       'Other Middle

In [32]:
countries = []

In [33]:
tabs_indicator_mapping1 = {
    'Primary Energy Consumption': to_concept_id('Primary Energy Consumption'),
    'Primary Energy - Cons capita': to_concept_id('Primary Energy Consumption per capita'),
    'Carbon Dioxide Emissions': to_concept_id('Carbon Dioxide Emissions'),
    'Oil - Refinery throughput': to_concept_id('Oil - Refinery throughput'),
    'Oil - Refining capacity': to_concept_id('Oil - Refining capacity'),
    'Oil - Proved reserves history': to_concept_id('Oil - Proved reserves'),
    'Gas - Proved reserves history ': to_concept_id('Gas - Proved reserves'),
    'Electricity Generation ': to_concept_id('Electricity Generation')
}

In [34]:
for t, indicator in tabs_indicator_mapping1.items():
    data = sheet_loader(source_file, sheet_name=t)
    data = preprocess(data)
    countries.append(data['geo_name'].unique())
    df = process_1(data, indicator)
    df.to_csv('by--geo--year.csv'.format(indicator), index=False)

In [35]:
def make_dict1(sheet, unit, fuel):
    return dict(sheet=sheet, unit=unit, fuel=fuel)

In [36]:
tabs_indicator_mapping2 = {
    'production': [
        make_dict1(sheet='Gas Production - Bcm', unit='bcm', fuel='gas'),
        make_dict1(sheet='Gas Production - Bcf', unit='bcf', fuel='gas'),
        # make_dict1('Gas Production - Mtoe', 'mtoe', 'gas'),
        make_dict1('Gas Production - EJ', 'exajoules', 'gas'),
        make_dict1('Coal Production - Tonnes', 'tonne', 'coal'),
        # make_dict1('Coal Production - Mtoe', 'mtoe', 'coal'),
        make_dict1('Coal Production - EJ', 'exajoules', 'coal'),
        
        #The Bio Fuels sheet has multiple tables in their sheet like BioDiesel, BioGasoline so they we created
        #another ETL biofuels_run.ipynb for it.
        
        #make_dict1('Biofuels Production - Kboed', 'kboed', 'biofuel'),
        #make_dict1('Biofuels Production - Ktoe', 'ktoe', 'biofuel'),
        make_dict1('Oil Production - Barrels', 'barrel', 'oil'),
        make_dict1('Oil Production - Tonnes', 'tonne', 'oil')
    ],
    'consumption': [
        make_dict1('Gas Consumption - Bcm', 'bcm', 'gas'),
        make_dict1('Gas Consumption - Bcf', 'bcf', 'gas'),
        #make_dict1('Gas Consumption - Mtoe', 'mtoe', 'gas'),
        make_dict1('Gas Consumption - EJ', 'exajoules', 'gas'),
        #make_dict1('Coal Consumption - Mtoe', 'mtoe', 'coal'),
        make_dict1('Coal Consumption - EJ', 'exajoules', 'coal'),
        #make_dict1('Nuclear Consumption - Mtoe', 'mtoe', 'nuclear'),
        make_dict1('Nuclear Consumption - EJ', 'exajoules', 'nuclear'),
        #make_dict1('Hydro Consumption - Mtoe', 'mtoe', 'hydro'),
        make_dict1('Hydro Consumption - EJ', 'exajoules', 'hydro'),
        # make_dict1('Other renewables - Mtoe', 'mtoe', 'other_renewables'),
        #make_dict1('Solar Consumption - Mtoe', 'mtoe', 'solar'),
        make_dict1('Solar Consumption - EJ', 'exajoules', 'solar'),
        #make_dict1('Wind Consumption - Mtoe', 'mtoe', 'wind'),
        make_dict1('Wind Consumption - EJ', 'exajoules', 'wind'),
        #make_dict1('Geo Biomass Other - Mtoe', 'mtoe', 'geo_biomass_other'),
        make_dict1('Geo Biomass Other - EJ', 'exajoules', 'geo_biomass_other'),
        make_dict1('Oil Consumption - Barrels', 'barrel', 'oil'),
        make_dict1('Oil Consumption - Tonnes', 'tonne', 'oil'),
        #make_dict1('Oil Consumption - Mtoe', 'mtoe', 'oil')
        make_dict1('Oil Consumption - EJ', 'exajoules', 'oil')
    ],
    'electricity_generation':[
        make_dict1('Elec Gen from Oil', 'twh', 'oil'),
        make_dict1('Elec Gen from Gas', 'twh', 'gas'),
        make_dict1('Elec Gen from Coal', 'twh', 'coal'),
        make_dict1('Elec Gen from Other', 'twh', 'other'),
        make_dict1('Nuclear Generation - TWh', 'twh', 'nuclear'),
        make_dict1('Hydro Generation - TWh', 'twh', 'hydro'),
        # make_dict1('Other renewables - TWh', 'twh', 'other_renewables'),
        make_dict1('Solar Generation - TWh', 'twh', 'solar'),
        make_dict1('Wind Generation -TWh', 'twh', 'wind'),
        make_dict1('Geo Biomass Other - TWh', 'twh', 'geo_biomass_other')
    ]
}

In [47]:
def preprocess_2(data):
    """preprocessing the data:
    1. rename the first column to geo_name
    2. rename the geo_name to alphanumeric
    3. drop all empty lines and lines after 'total world'

    Note: This function only applies to the tab with country as row index
    and year as column index.
    """
    data = data.rename(columns={data.columns[0]: 'geo_name'})
    data['geo'] = data['geo_name'].map(to_concept_id)
    data = data.set_index('geo')
    data = data.dropna(how='all')
    if 'world' in data.index:
        data = data.loc[:'world']
    else:
        data = data.loc[:'total_world']
    data = data.reset_index()
    return data
def process_2(data, ddf_id):
    data = data.dropna(axis=1, how='all')
    data = data.drop('geo_name', axis=1)
    idx = list(data.columns).index(2019)  # TODO
    data = data.iloc[:, :idx + 1]  # drop columns after latest year of each sheet.
    #data = data.drop(['2017.1', '2017.2', '2006-16'], axis=1)
    
    data = data.set_index('geo')

    d = data.T.unstack()
    d = d.dropna()
    d = d.reset_index()
    d.columns = ['geo', 'year', ddf_id]

    return d.sort_values(by=['geo', 'year'])

In [37]:
for i, ms in tabs_indicator_mapping2.items():
    #Hisham: I made this datapoint indicator--by--geo--year.csv
    #Previously it was datapoints--indicator--by--geo--fuel--year--unit.csv
    for m in ms:
        data = []
        d = sheet_loader(source_file, sheet_name=m['sheet'])
        d = preprocess(d)        
        countries.append(d['geo_name'].unique())
        indicator_name = m['fuel'] + '_' + i + '_' + m['unit']
        d = process_1(d, indicator_name)        
        #d['fuel'] = m['fuel']
        #d['unit'] = m['unit']
        data.append(d)
        data = pd.concat(data, ignore_index=True)
        df = data[['geo', 'year', indicator_name]]    
        df[indicator_name] = df[indicator_name].map(number_formatter)
        df = df.sort_values(by=['geo', 'year'])

In [39]:
df.to_csv('by--geo--year.csv'.format(m['fuel'] + '_' + i + '_' + m['unit']), index=False)

In [40]:
countries2 = []

In [44]:
def create_datapoint_2(data_tabs, fuels, indicator):
    data = []

    for t, f in zip(data_tabs, fuels):
        d = sheet_loader(source_file, sheet_name=t)
        d = preprocess_2(d)
        countries2.append(d['geo_name'].unique())
        d = process_2(d, indicator)
        d['material'] = f
        data.append(d)
    data = pd.concat(data, ignore_index=True)
    df = data[['geo', 'material', 'year', indicator]]
    df[indicator] = df[indicator].map(number_formatter)
    df.to_csv('by--geo--mineral--year.csv'.format(indicator), index=False)
    
    return df

In [42]:
data_tabs = ['Cobalt Production-Reserves', 
             'Lithium Production-Reserves',
             'Graphite Production-Reserves',
             'Rare Earth Production-Reserves'
            ]
fuels  = ['cobalt', 'lithium', 'graphite', 'rare_earth']
indicator = 'production_reserve'

In [48]:
df = create_datapoint_2(data_tabs, fuels, indicator)

In [51]:
countries2 = np.concatenate(countries2)

In [52]:
c2 = pd.DataFrame({'name': countries2})

In [53]:
c2['name'] = c2['name'].str.strip()

In [54]:
c2 = c2.drop_duplicates(subset='name')

In [55]:
c2

,name
0,Australia
1,Canada
2,DR Congo
3,Cuba
4,Madagascar
5,Morocco
6,New Caledonia
7,Papua New Guinea
8,Philippines
9,Russian Federation


In [56]:
df.loc[df.geo == 'brazil1', 'geo'] = 'brazil'
df.loc[df.geo == 'india2', 'geo'] = 'india'
df.loc[df.geo == 'rest_of_world2', 'geo'] = 'rest_of_world'
df.loc[df.geo == 'rest_of_world3', 'geo'] = 'rest_of_world'

In [57]:
df.loc[df.geo == 'world', 'geo'] = 'total_world'

In [59]:
df.sample(20)

,geo,material,year,production_reserve
72,cuba,cobalt,2017,5
874,australia,rare_earth,2007,0
317,argentina,lithium,1995,0.008
878,australia,rare_earth,2011,2.188
1022,thailand,rare_earth,1997,0.012
1015,russian_federation,rare_earth,2015,2.31230005
242,south_africa,cobalt,1995,0.288
539,us,lithium,2019,0.9
848,zimbabwe,graphite,1995,11.381
1058,total_world,rare_earth,2019,209.599


In [60]:
(df.sort_values(by=['material', 'geo', 'year']).to_csv('production_reserve--by--geo--material--year.csv', index=False))

In [61]:
sheet_loader2 = partial(pd.read_excel, skiprows=3)

In [62]:
countries3 = []

In [63]:
def create_datapoint_3(data_tabs, fuels, indicator):
    data = []

    for t, f in zip(data_tabs, fuels):
        d = sheet_loader2(source_file, sheet_name=t)
        d = preprocess_2(d)
        countries3.append(d['geo_name'].unique())
        d = process_2(d, indicator)
        d['renewable'] = f
        data.append(d)
    data = pd.concat(data, ignore_index=True)
    df = data[['geo', 'renewable', 'year', indicator]]
    df[indicator] = df[indicator].map(number_formatter)
    
    
    return df

In [64]:
data_tabs = ['Geothermal Capacity', 
             'Solar Capacity',
             'Wind Capacity'
            ]
fuels  = ['geothermal', 'solar', 'wind']
indicator = 'installed_capacity'

In [65]:
df = create_datapoint_3(data_tabs, fuels, indicator)

In [67]:
df.sample(20)

,geo,renewable,year,installed_capacity
3245,total_europe,wind,2011,96439.363
1899,total_europe,solar,1996,66.9
881,bulgaria,solar,2010,25
1739,sweden,solar,2004,4
3173,total_africa,wind,2005,226.38
1533,pakistan,solar,2014,165.245
968,czech_republic,solar,2001,0.1
578,total_asia_pacific,geothermal,2006,3719.6
1342,morocco,solar,2015,199.8
768,algeria,solar,2017,425


In [68]:
(df.sort_values(by=['renewable', 'geo', 'year'])
 .to_csv('by--geo--renewable--year.csv'.format(indicator), index=False))

### https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html